# Week 4 - Models and Experimentation

## Step 1 Training a model

### Install packages

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

### Import data

We will be using Diamonds dataset imported from Seaborn. It is also available on [Kaggle](https://www.kaggle.com/datasets/shivam2503/diamonds).

Read about the features by following the link. We will be predicting the price of diamonds.

In [2]:
diamonds = sns.load_dataset("diamonds")
diamonds.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [3]:
diamonds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   carat    53940 non-null  float64 
 1   cut      53940 non-null  category
 2   color    53940 non-null  category
 3   clarity  53940 non-null  category
 4   depth    53940 non-null  float64 
 5   table    53940 non-null  float64 
 6   price    53940 non-null  int64   
 7   x        53940 non-null  float64 
 8   y        53940 non-null  float64 
 9   z        53940 non-null  float64 
dtypes: category(3), float64(6), int64(1)
memory usage: 3.0 MB


In [4]:
diamonds.shape

(53940, 10)

In [5]:
X, y = diamonds.drop("price", axis=1), diamonds[["price"]]

# For the cut, color and clarity use pandas category to enable XGBoost ability to deal with categorical data.
X["cut"] = X["cut"].astype("category")
X["color"] = X["color"].astype("category")
X["clarity"] = X["clarity"].astype("category")

### Split the data and train a model

In [6]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Create DMatrix
dtrain = xgb.DMatrix(X_train, label=y_train, enable_categorical=True)
dtest = xgb.DMatrix(X_test, label=y_test, enable_categorical=True)

In [7]:
# Define hyperparameters
params = {"objective": "reg:squarederror", "tree_method": "hist"}

n = 100
model = xgb.train(
    params=params,
    dtrain=dtrain,
    num_boost_round=n,
)

In [8]:
# Define evaluation metrics - Root Mean Squared Error

predictions = model.predict(dtest)
rmse = mean_squared_error(y_test, predictions, squared=False)
print(f"RMSE: {rmse}")

RMSE: 545.191877397669


/Users/jacob/.pyenv/versions/miniconda3-3.11-24.1.2-0/envs/practicum/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


### Incorporate validation

In [9]:
params = {"objective": "reg:squarederror", "tree_method": "hist"}
n = 100

# Create the validation set
evals = [(dtrain, "train"), (dtest, "validation")]

In [10]:
evals = [(dtrain, "train"), (dtest, "validation")]

model = xgb.train(
    params=params,
    dtrain=dtrain,
    num_boost_round=n,
    evals=evals,
    verbose_eval=10,
)

[0]	train-rmse:2861.71326	validation-rmse:2853.85688
[10]	train-rmse:554.29819	validation-rmse:579.26422
[20]	train-rmse:493.68077	validation-rmse:547.75493
[30]	train-rmse:467.32713	validation-rmse:540.03567
[40]	train-rmse:447.40974	validation-rmse:541.70531
[50]	train-rmse:432.62075	validation-rmse:540.89769
[60]	train-rmse:422.28318	validation-rmse:540.63039
[70]	train-rmse:410.72350	validation-rmse:543.67077
[80]	train-rmse:398.24619	validation-rmse:545.08296
[90]	train-rmse:386.92486	validation-rmse:543.90036
[99]	train-rmse:379.58717	validation-rmse:545.19188


In [11]:
# Incorporate early stopping
n = 10000


model = xgb.train(
    params=params,
    dtrain=dtrain,
    num_boost_round=n,
    evals=evals,
    verbose_eval=50,
    # Activate early stopping
    early_stopping_rounds=50,
)

[0]	train-rmse:2861.71326	validation-rmse:2853.85688
[50]	train-rmse:432.62075	validation-rmse:540.89769
[82]	train-rmse:394.92609	validation-rmse:544.73937


In [12]:
# Cross-validation

params = {"objective": "reg:squarederror", "tree_method": "hist"}
n = 1000

results = xgb.cv(params, dtrain, num_boost_round=n, nfold=5, early_stopping_rounds=20)

In [13]:
results.head()

,train-rmse-mean,train-rmse-std,test-rmse-mean,test-rmse-std
0,2861.512810,8.494816,2861.704341,37.144992
1,2081.847733,5.811005,2084.838207,31.889208
2,1547.031906,5.092391,1554.657450,30.699908
3,1184.129738,3.982239,1194.251600,26.940062
4,942.998782,3.327174,960.239319,24.392689


In [14]:
best_rmse = results["test-rmse-mean"].min()

best_rmse

553.4613038243663

## Start W&B


- Login into your W&B profile using the code below
- Alternatively you can set environment variables. There are several env variables which you can set to change the behavior of W&B logging. The most important are:
    - WANDB_API_KEY - find this in your "Settings" section under your profile
    - WANDB_BASE_URL - this is the url of the W&B server

- Find your API Token in "Profile" -> "Setttings" in the W&B App



In [15]:
# Log in to your W&B account
import wandb

wandb.login()

wandb: Currently logged in as: jacobnu. Use `wandb login --relogin` to force relogin


True

We're going to perform a grid search using sweep. ([Source](https://docs.wandb.ai/guides/sweeps/walkthrough))

In [16]:
# performing a grid search
sweep_params = {
    "method": "grid",
    "metric": {"name": "rmse", "goal": "minimize"},
    "parameters": {
        "learning_rate": {"values": [0.1, 0.15, 0.2]},
        "subsample": {"values": [0.7, 0.8, 0.9]},
        "max_depth": {"values": [5, 7, 9]},
        "n_estimators": {"values": [100, 200]},
    },
}

In [17]:
sweep_id = wandb.sweep(
    sweep_params,
    project="xbg_grid_diamonds",
    entity="jacobnu",
)

Create sweep with ID: hcbnoalh
Sweep URL: https://wandb.ai/jacobnu/xbg_grid_diamonds/sweeps/hcbnoalh


In [18]:
def main():
    run = wandb.init(project="xbg_grid_diamonds", entity="jacobnu", config=wandb.config)

    # Define the search space
    xgb_params = {
        "objective": "reg:squarederror",
        "learning_rate": run.config.learning_rate,
        "subsample": run.config.subsample,
        "max_depth": int(run.config.max_depth),
        "n_estimators": int(run.config.n_estimators),
    }

    evals_result = {}

    # Train XGB
    model = xgb.train(
        xgb_params,
        dtrain,
        num_boost_round=run.config.n_estimators,
        evals=[(dtest, "Test")],
        early_stopping_rounds=10,
        evals_result=evals_result,
    )

    # Log the best score
    rmse = model.best_score
    wandb.log({"rmse": rmse})

    run.finish()

In [19]:
wandb.agent(sweep_id, function=main)

wandb: Agent Starting Run: asyoviih with config:
wandb: 	learning_rate: 0.1
wandb: 	max_depth: 5
wandb: 	n_estimators: 100
wandb: 	subsample: 0.7
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


[0]	Test-rmse:3614.08143
[1]	Test-rmse:3277.95006
[2]	Test-rmse:2977.83384
[3]	Test-rmse:2709.54296
[4]	Test-rmse:2469.74865
[5]	Test-rmse:2253.70434
[6]	Test-rmse:2060.39691
[7]	Test-rmse:1887.31711
[8]	Test-rmse:1734.77591
[9]	Test-rmse:1595.33099
[10]	Test-rmse:1473.18562
[11]	Test-rmse:1362.73455
[12]	Test-rmse:1266.75866
[13]	Test-rmse:1181.65248
[14]	Test-rmse:1105.81547
[15]	Test-rmse:1036.32919
[16]	Test-rmse:976.33722
[17]	Test-rmse:921.83507
[18]	Test-rmse:876.67865
[19]	Test-rmse:836.93122
[20]	Test-rmse:801.42868
[21]	Test-rmse:769.76047
[22]	Test-rmse:743.37744
[23]	Test-rmse:721.04846
[24]	Test-rmse:701.60432
[25]	Test-rmse:682.18818
[26]	Test-rmse:665.66599
[27]	Test-rmse:652.72696
[28]	Test-rmse:640.52368
[29]	Test-rmse:631.68027
[30]	Test-rmse:623.22869
[31]	Test-rmse:614.88724
[32]	Test-rmse:607.93096
[33]	Test-rmse:601.94250
[34]	Test-rmse:595.61825
[35]	Test-rmse:591.38973
[36]	Test-rmse:587.12918
[37]	Test-rmse:582.32684
[38]	Test-rmse:578.50656
[39]	Test-rmse:575.

[21:19:12] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "n_estimators" } are not used.



[74]	Test-rmse:541.51627
[75]	Test-rmse:541.61863
[76]	Test-rmse:541.64863
[77]	Test-rmse:541.48134
[78]	Test-rmse:541.49342
[79]	Test-rmse:541.14853
[80]	Test-rmse:540.84747
[81]	Test-rmse:540.72672
[82]	Test-rmse:540.74962
[83]	Test-rmse:541.01296
[84]	Test-rmse:540.70914
[85]	Test-rmse:540.26195
[86]	Test-rmse:540.06617
[87]	Test-rmse:540.02049
[88]	Test-rmse:540.19586
[89]	Test-rmse:539.93972
[90]	Test-rmse:539.95317
[91]	Test-rmse:539.65053
[92]	Test-rmse:539.60620
[93]	Test-rmse:539.64990
[94]	Test-rmse:539.92832
[95]	Test-rmse:539.54783
[96]	Test-rmse:539.23616
[97]	Test-rmse:538.96280
[98]	Test-rmse:538.88757
[99]	Test-rmse:538.59013


rmse,▁
rmse,538.59013


...

[0]	Test-rmse:3217.72448
[1]	Test-rmse:2605.94856
[2]	Test-rmse:2119.55007
[3]	Test-rmse:1734.49103
[4]	Test-rmse:1433.49192
[5]	Test-rmse:1198.24163
[6]	Test-rmse:1016.43841
[7]	Test-rmse:881.57935
[8]	Test-rmse:780.43988
[9]	Test-rmse:707.51512
[10]	Test-rmse:653.43786
[11]	Test-rmse:616.43555
[12]	Test-rmse:590.19956
[13]	Test-rmse:574.10839
[14]	Test-rmse:564.34365
[15]	Test-rmse:556.62377
[16]	Test-rmse:552.16175
[17]	Test-rmse:549.13331
[18]	Test-rmse:546.50464
[19]	Test-rmse:544.01666
[20]	Test-rmse:542.59967
[21]	Test-rmse:541.98918
[22]	Test-rmse:540.33207
[23]	Test-rmse:540.64779
[24]	Test-rmse:540.70651
[25]	Test-rmse:541.26678
[26]	Test-rmse:539.91871
[27]	Test-rmse:539.47799
[28]	Test-rmse:540.73590
[29]	Test-rmse:540.98115
[30]	Test-rmse:541.59364
[31]	Test-rmse:541.51475
[32]	Test-rmse:542.70807
[33]	Test-rmse:542.75032
[34]	Test-rmse:543.22498
[35]	Test-rmse:543.88259
[36]	Test-rmse:544.27206


[21:30:38] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "n_estimators" } are not used.



[37]	Test-rmse:544.35725


rmse,▁
rmse,539.47799


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


In [37]:
eval_df = pd.read_csv("wandb_runs.csv")

In [39]:
eval_df.sort_values(by=['rmse'], ascending=True).head(10)

,Name,Created,Runtime,learning_rate,max_depth,n_estimators,subsample,rmse
7,wandering-sweep-11,2024-04-22T02:20:53.000Z,8,0.10,7,200,0.8,527.266012
8,volcanic-sweep-8,2024-04-22T02:20:22.000Z,7,0.10,7,100,0.8,527.266012
10,warm-sweep-7,2024-04-22T02:20:12.000Z,7,0.10,7,100,0.7,530.268757
9,icy-sweep-10,2024-04-22T02:20:43.000Z,7,0.10,7,200,0.7,530.268757
12,dry-sweep-25,2024-04-22T02:23:21.000Z,12,0.15,7,100,0.7,532.398341
11,vivid-sweep-28,2024-04-22T02:23:57.000Z,8,0.15,7,200,0.7,532.398341
13,ancient-sweep-6,2024-04-22T02:20:02.000Z,8,0.10,5,200,0.9,532.516888
15,vital-sweep-26,2024-04-22T02:23:36.000Z,7,0.15,7,100,0.8,532.576118
14,chocolate-sweep-29,2024-04-22T02:24:07.000Z,8,0.15,7,200,0.8,532.576118
17,hardy-sweep-13,2024-04-22T02:21:14.000Z,7,0.10,9,100,0.7,532.690671


## Observations

1. **Best and Worst Performers**:
   The lowest RMSE scores, which indicate the best model performance, are achieved in the configurations `wandering-sweep-11` and `volcanic-sweep-8`, both with an RMSE of approximately 527.27. These runs used a learning rate of 0.1, `max_depth` of 7, `n_estimators` of 200 and 100, respectively, and a `subsample` of 0.8.
   - The highest RMSE scores, indicating poorer performance, are observed in `lemon-sweep-52` and `glad-sweep-49` with an RMSE of around 545.29. These settings included a learning rate of 0.2, `max_depth` of 9, `n_estimators` of 200 and 100, respectively, and a `subsample` of 0.7.
2. **Impact of Learning Rate**:
   - Lower learning rates (0.1) paired with mid-range `max_depth` (7) and higher `n_estimators` (200) tend to perform better. This suggests that slower learning and more estimators allow for more robust generalization in this dataset.
   - Higher learning rates (0.2), especially combined with higher `max_depth` and lower `n_estimators`, lead to worse performance, possibly due to overfitting or insufficient learning iterations.
3. **Influence of Subsampling**:
   - A subsample ratio of 0.8 consistently appears in many of the better-performing configurations, indicating that this subsampling level helps achieve a good balance between training speed and model accuracy by reducing variance and preventing overfitting.
4. **Effect of Max Depth and Estimators**:
   - Moderate `max_depth` values (7) often yield better results than very high (9) or lower (5) depths. This indicates that too-deep trees might be overfitting, while too-shallow trees might be underfitting.
   - Increasing `n_estimators` from 100 to 200 generally improves performance, suggesting that more boosting rounds are beneficial up to a point. However, further increasing `n_estimators` beyond a certain threshold might result in diminishing returns and increased computational cost without significant gains in performance.